In [1]:
import sys
from pyspark.sql import SparkSession, functions, types, Row
import re
from pyspark.sql.functions import lit
import numpy as np
import math
spark = SparkSession.builder.appName('correlate logs').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
assert spark.version >= '2.3' # make sure we have Spark 2.3+

line_re = re.compile(r"^(\S+) - - \[\S+ [+-]\d+\] \"[A-Z]+ \S+ HTTP/\d\.\d\" \d+ (\d+)$")


def line_to_row(line):
    """
    Take a logfile line and return a Row object with hostname and bytes transferred. Return None if regex doesn't match.
    """
    m = line_re.match(line)
    if m:
        # TODO
        return Row(hostname=m.group(1),bytes=float(m.group(2)))
    else:
        return None

def not_none(row):
    """
    Is this None? Hint: .filter() with it.
    """
    return row is not None


def create_row_rdd(in_directory):
    log_lines = spark.sparkContext.textFile(in_directory)
    log_lines = log_lines.map(line_to_row)
    log_lines = log_lines.filter(not_none)
    return log_lines
    # TODO: return an RDD of Row() objects

In [2]:

def main(in_directory):
    logs = spark.createDataFrame(create_row_rdd(in_directory))
    logs.show()
    count_requests = logs.groupby('hostname').count().cache()
    sum_requests_bytes = logs.groupby('hostname').sum().cache()
    data = sum_requests_bytes.join(count_requests,'hostname')
#     data.show()
    # TODO: calculate r.
    data = data.withColumn('1',lit(1))
    data = data.withColumnRenamed('count','xi')
    data = data.withColumnRenamed('sum(bytes)','yi')
    data = data.withColumn('xi2',data['xi']*data['xi'])
    data = data.withColumn('yi2',data['yi']*data['yi'])
    data = data.withColumn('xiyi',data['xi']*data['yi'])
    data.show()
    group_data = data.groupby().sum().cache()
    value = group_data.first()
    n = value[2]
    sum_xi = value[1]
    sum_yi = value[0]
    sum_xi2 = value[3]
    sum_yi2 = value[4]
    sum_xiyi = value[5]
    r = (n*sum_xiyi - sum_xi*sum_yi)/(math.sqrt(n*sum_xi2 - sum_xi**2)*math.sqrt(n*sum_yi2 - sum_yi**2))
    print("r = %g\nr^2 = %g" % (r, r**2))


if __name__=='__main__':
    in_directory = sys.argv[1]
    main(in_directory)


Py4JJavaError: An error occurred while calling o28.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/zelinh/CMPT353/e11/-f
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(